In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')

In [ ]:
df.head()

**First We check which columns are important or which are not important for our model.look at the rows we found that our target attribute (target attribute predict that a person looking for a new job or work for our company) it's not depended on the enrollment no and city also so we drop that columns**

In [ ]:
df_preproceesing = df.drop(['enrollee_id','city'],axis=1)

In [ ]:
df_preproceesing.info()

Now We preprocess our data.

1.handle missing values

In [ ]:
print(pd.DataFrame(df_preproceesing.isnull().sum()/len(df_preproceesing)*100,columns=['missing values in %']))

In [ ]:
df_object_columns = list(df_preproceesing.select_dtypes(['object']).columns)
df_object_columns_unique_value = [list(df_preproceesing[i].unique()) for i in df_object_columns]

These all are unique values which are presents in all categorical features.please ingone the nan value<br><br>
[['Male', nan, 'Female', 'Other'],
 ['Has relevent experience', 'No relevent experience'],
 ['no_enrollment', 'Full time course', nan, 'Part time course'],
 ['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'],
 ['STEM', 'Business Degree', nan, 'Arts', 'Humanities', 'No Major', 'Other'],
 ['>20',
  '15',
  '5',
  '<1',
  '11',
  '13',
  '7',
  '17',
  '2',
  '16',
  '1',
  '4',
  '10',
  '14',
  '18',
  '19',
  '12',
  '3',
  '6',
  '9',
  '8',
  '20',
  nan],
 [nan,
  '50-99',
  '<10',
  '10000+',
  '5000-9999',
  '1000-4999',
  '10/49',
  '100-500',
  '500-999'],
 [nan,
  'Pvt Ltd',
  'Funded Startup',
  'Early Stage Startup',
  'Other',
  'Public Sector',
  'NGO'],
 ['1', '>4', 'never', '4', '3', '2', nan]]

In [ ]:
def piechart(column_name,labels):
    plt.figure(figsize=(8,8))
    plt.pie(df_preproceesing[column_name].value_counts(),shadow=True,labels=labels,autopct = '%1.2f%%')

In [ ]:
piechart('relevent_experience',['Has relevent experience', 'No relevent experience'])

we assume that in gender feature all nan values are treat as Other category and same as that we do with others categories too

In [ ]:
df_preproceesing['gender'] = df_preproceesing['gender'].fillna("Other")
df_preproceesing['relevent_experience'] = df_preproceesing['relevent_experience'].fillna("No relevent experience")
df_preproceesing['enrolled_university'] = df_preproceesing['enrolled_university'].fillna("no_enrollment")
df_preproceesing['experience'] = df_preproceesing['experience'].fillna("<1")
df_preproceesing['company_size'] = df_preproceesing['company_size'].fillna("<10")
df_preproceesing['last_new_job'] = df_preproceesing['last_new_job'].fillna("never")

Basically if we have categorical feature with Nan value then we fill them with mode of that feature

and if we have numrical feature than we first check distrubation if there is normal distrubation then we fill with mean otherwise we fill missing value with median

This is basically fill a mode value at missing place

In [ ]:
for i in df_object_columns:
    m =  df_preproceesing[i].mode()[0]
    df_preproceesing[i] = df_preproceesing[i].fillna(m)

we handled missing value lets see that

In [ ]:
df_preproceesing.isnull().sum()

Now we check number of increment or decrement in particular categorie

In [ ]:
fig, axs = plt.subplots(nrows=8,ncols=2, figsize=(15,30))
sns.countplot(x="relevent_experience",hue='target', data=df, ax=axs[0,0])
sns.countplot(x="relevent_experience",hue='target', data=df_preproceesing, ax=axs[0,1])
sns.countplot(x="enrolled_university",hue='target', data=df, ax=axs[1,0])
sns.countplot(x="enrolled_university",hue='target', data=df_preproceesing, ax=axs[1,1])
sns.countplot(x="education_level",hue='target', data=df, ax=axs[2,0])
sns.countplot(x="education_level",hue='target', data=df_preproceesing, ax=axs[2,1])
sns.countplot(x="major_discipline",hue='target', data=df, ax=axs[3,0])
sns.countplot(x="major_discipline",hue='target', data=df_preproceesing, ax=axs[3,1])
sns.countplot(x="experience",hue='target', data=df, ax=axs[4,0])
sns.countplot(x="experience",hue='target', data=df_preproceesing, ax=axs[4,1])
sns.countplot(x="company_size",hue='target', data=df, ax=axs[5,0])
sns.countplot(x="company_size",hue='target', data=df_preproceesing, ax=axs[5,1])
sns.countplot(x="company_type",hue='target', data=df, ax=axs[6,0])
sns.countplot(x="company_type",hue='target', data=df_preproceesing, ax=axs[6,1])
sns.countplot(x="last_new_job",hue='target', data=df, ax=axs[7,0])
sns.countplot(x="last_new_job",hue='target', data=df_preproceesing, ax=axs[7,1])

Now we encode our data using ordinalencoder or onehotencoding

In [ ]:
for i in df_object_columns:
    print(f"{i} : {len(df_preproceesing[i].unique())} categories")

<h3>First we encoding all ordinal categorical variable using this code</h3><br><br>
(ordinal means a categorical value with an order like a good,bad,andvery good that means we can assume that a bad is like 0 , good is like 1, and very good is like 2)

In [ ]:
gender_map = {
    "Male": 0,
    "Female": 1,
    "Other": 2
}
relevent_experience_map = {
    "No relevent experience": 0,
    "Has relevent experience": 1
}
enrolled_university_map = {
    "no_enrollment": 0,
    "Full time course": 2,
    "Part time course": 1
}
education_level_map = {
    "High School": 1,
    "Primary School" :0,
    "Phd":4, 
    "Masters": 3,
    "Graduate": 2
}
major_discipline_map = {
    "STEM": 0,
    "Humanities": 1,
    "Other": 2,
    "Business Degree": 3,
    "Arts": 4,
    "No Major": 5
}
experience_map = {
    "<1": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10,
    "11": 11,
    "12": 12,
    "13": 13,
    "14": 14,
    "15": 15,
    "16": 16,
    "17": 17,
    "18": 18,
    "19": 19,
    "20": 20,
    ">20": 21,
}
company_size_map = {
    "<10": 0,
    "10/49": 1,
    "50-99": 2,
    "100-500": 3,
    "500-999": 4,
    "1000-4999": 5,
    "5000-9999": 6,
    "10000+": 7
}
company_type_map = {
    "Pvt Ltd": 0,
    "Funded Startup": 1,
    "Public Sector": 2,
    "Early Stage Startup": 3,
    "NGO": 4,
    "Other": 5,
}
last_new_job_map = {
    "never": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    ">4": 5,
}

In [ ]:
df_preproceesing.gender = df_preproceesing.gender.map(gender_map)
df_preproceesing.relevent_experience = df_preproceesing.relevent_experience.map(relevent_experience_map)
df_preproceesing.enrolled_university = df_preproceesing.enrolled_university.map(enrolled_university_map)
df_preproceesing.education_level = df_preproceesing.education_level.map(education_level_map)
df_preproceesing.major_discipline = df_preproceesing.major_discipline.map(major_discipline_map)
df_preproceesing.experience = df_preproceesing.experience.map(experience_map)
df_preproceesing.company_size = df_preproceesing.company_size.map(company_size_map)
df_preproceesing.company_type = df_preproceesing.company_type.map(company_type_map)
df_preproceesing.last_new_job = df_preproceesing.last_new_job.map(last_new_job_map)

In [ ]:
df_preproceesing.head()

Now we split our data in train test ans also do a scalling

In [ ]:
x = df_preproceesing.iloc[:,:-1]
y = df_preproceesing.iloc[:,-1]

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sc = StandardScaler()

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
x = pd.DataFrame(sc.fit_transform(x),columns=['city_development_index','gender','relevent_experience','enrolled_university','education_level','major_discipline','experience','company_size','company_type','last_new_job','training_hours'])

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

<h2>important message:- </h2>
In this data i were only focused on just data cleaning ot data analysis dont mind about the low accuracy.int next project i will do one step ahed like which model best for our data which one give us best accuracy etc.